In [1]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import logging
import json
from pathlib import Path


from wavlm.WavLM import WavLM, WavLMConfig
from hifigan.models import Generator as HiFiGAN
from hifigan.utils import AttrDict
from matcher import ExNOTVC
import torchaudio

import random
import numpy as np

from IPython.display import clear_output
from tqdm.notebook import tqdm as tqdm

DEVICE = 'cuda'

In [2]:
def xnot_vc(pretrained=True, progress=True, prematched=False, device='cuda') -> ExNOTVC:
    """ Load kNN-VC (WavLM encoder and HiFiGAN decoder). Optionally use vocoder trained on `prematched` data. """
    hifigan, hifigan_cfg = hifigan_wavlm(pretrained, progress, prematched, device)
    wavlm = wavlm_large(pretrained, progress, device)
    xnotvc = ExNOTVC(wavlm, hifigan, hifigan_cfg, device)
    return xnotvc


def hifigan_wavlm(pretrained=True, progress=True, prematched=False, device='cuda') -> HiFiGAN:
    """ Load pretrained hifigan trained to vocode wavlm features. Optionally use weights trained on `prematched` data. """
    #cp = Path(__file__).parent.absolute()

    with open('hifigan/config_v1_wavlm.json') as f:
        data = f.read()
    json_config = json.loads(data)
    h = AttrDict(json_config)
    device = torch.device(device)

    generator = HiFiGAN(h).to(device)
    
    if pretrained:
        if prematched:
            url = "https://github.com/bshall/knn-vc/releases/download/v0.1/prematch_g_02500000.pt"
        else:
            print("Загружаем непреметченный")
            url = "https://github.com/bshall/knn-vc/releases/download/v0.1/g_02500000.pt"
        state_dict_g = torch.hub.load_state_dict_from_url(
            url,
            map_location=device,
            progress=progress
        )
        generator.load_state_dict(state_dict_g['generator'])
    generator.eval()
    generator.remove_weight_norm()
    print(f"[HiFiGAN] Generator loaded with {sum([p.numel() for p in generator.parameters()]):,d} parameters.")
    return generator, h


def wavlm_large(pretrained=True, progress=True, device='cuda') -> WavLM:
    """Load the WavLM large checkpoint from the original paper. See https://github.com/microsoft/unilm/tree/master/wavlm for details. """
    if torch.cuda.is_available() == False:
        if str(device) != 'cpu':
            logging.warning(f"Overriding device {device} to cpu since no GPU is available.")
            device = 'cpu'
    checkpoint = torch.hub.load_state_dict_from_url(
        "https://github.com/bshall/knn-vc/releases/download/v0.1/WavLM-Large.pt", 
        map_location=device, 
        progress=progress
    )
    
    cfg = WavLMConfig(checkpoint['cfg'])
    device = torch.device(device)
    model = WavLM(cfg)
    if pretrained:
        model.load_state_dict(checkpoint['model'])
    model = model.to(device)
    model.eval()
    print(f"WavLM-Large loaded with {sum([p.numel() for p in model.parameters()]):,d} parameters.")
    return model


In [3]:

xnotvc = xnot_vc() #загружаем веса для WavLM, HiFi Gan, убираем флажок преметчинга 


/home/wlad/projects/smiles/xnot-vc/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Загружаем непреметченный
Removing weight norm...
[HiFiGAN] Generator loaded with 16,523,393 parameters.
WavLM-Large loaded with 315,453,120 parameters.


### Пути к аудиодорожкам с женским голосом и мужским

Domain female - домен с женским голосом, Domain male - домен с мужским голосом. Мы хотим из женского голоса сделать мужской

In [4]:
domain_female_path = "data/female_3112.flac"
domain_male_path = "data/male.flac"


### Кодируем аудиодорожки с помощью WavLM 

In [5]:
features_female = xnotvc.get_features(domain_female_path)
features_male = xnotvc.get_features(domain_male_path)

WavLM каждые ~20-30 мс звуковой дорожки представляет в виде вектора фичей размерностью 1024

In [6]:
print(features_female.shape) #[длина дорожки делить на ~20-30мс , 1024] = массив из 7971 векторов размерностью 1024

torch.Size([7971, 1024])


In [7]:
print(features_male.shape) #массив из 5473 векторов размерностью 1024

torch.Size([5473, 1024])


### Sampler from voice distributions

Случайным образом выбираем батч векторов фичей

In [8]:
print(features_female.shape)

torch.Size([7971, 1024])


In [9]:
def sample_from_tensor(t, batch_size = 64):
    tensor = t.detach().cpu().numpy()
    batch = np.empty((batch_size, 1024))
    for i in range(batch_size):
        indice = random.randrange(0, tensor.shape[0])
        sample_tensor = tensor[indice]
        batch[i]=sample_tensor
    return batch.astype(np.float32)

In [10]:
sample_from_tensor(features_female).shape

(64, 1024)

In [11]:
sample_from_tensor(features_male).shape

(64, 1024)

### Функция стоимости

In [12]:
def sq_cost(X,Y):
    return (X-Y).square().flatten(start_dim=1).mean(dim=1)

COST = sq_cost

### Define NNs for transport map ***T*** : R^1024 -> R^1024 (generator) and potential ***f*** : R^1024 -> R (discriminator).
Medium size multilayer perceptrons with ReLU.

In [13]:
class NegAbs(nn.Module):
    def __init__(self):
        super(NegAbs, self).__init__()

    def forward(self, input):
        return -torch.abs(input)

T = nn.Sequential(
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024)
).to(DEVICE)

f = nn.Sequential(
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1),
    NegAbs(),
).to(DEVICE)

def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()


print('T params:', np.sum([np.prod(p.shape) for p in T.parameters()]))
print('f params:', np.sum([np.prod(p.shape) for p in f.parameters()]))

T params: 8396800
f params: 7348225


Оптимизаторы

In [14]:

T_opt = torch.optim.Adam(T.parameters(), lr=1e-4, weight_decay=1e-10)
f_opt = torch.optim.Adam(f.parameters(), lr=1e-4, weight_decay=1e-10)

In [15]:
#Parameters
T_ITERS = 10 # T updates per 1 f update
MAX_ITERS = 40001

to_torch = lambda x: torch.Tensor(x).to(DEVICE)

In [16]:
X = to_torch(sample_from_tensor(features_female))
Y = to_torch(sample_from_tensor(features_male))

In [17]:
sq_cost(X,Y)

tensor([12.9364, 14.1963, 14.2252, 16.4632, 14.7251, 17.9669, 10.9622,  9.8912,
        11.6634, 13.4767,  7.7740, 12.1300, 14.4305, 15.0345, 15.5719, 13.1378,
        14.6850, 14.5382, 11.7163, 13.5697, 15.4074, 19.0393, 14.1454, 15.8891,
        12.1140, 14.0060, 17.3926, 11.5281, 19.4218, 14.4317, 14.9812, 12.4851,
         8.9463, 12.0373, 10.9515, 12.6552, 13.0478, 10.4289, 15.8846, 17.8424,
        13.5686, 12.6834, 13.4447,  9.7368, 15.9797, 14.7639, 13.8420, 15.0298,
        13.8156, 15.5387, 16.6647,  9.1825, 15.1093, 11.6998, 13.3185, 15.4327,
        13.8612, 16.4122, 16.2840, 11.4705, 14.4066, 14.9373, 16.5880,  9.2981],
       device='cuda:0')

## Обучение

Сбрасываем веса

In [18]:
weight_reset(T); weight_reset(f)

Параметр W из алгоритма неполного транспорта

In [19]:
W=16

Алгоритм неполного оптимального транспорта

In [20]:
#XNOT algo

for step in tqdm (range(MAX_ITERS)):
    T_loss_ar = []
    f_loss_ar = []
    #T optimization
    T.train(True); f.eval()
    for t_iter in range(T_ITERS):
        X = to_torch(sample_from_tensor(features_female)) 
        T_loss = COST(X, T(X)).mean() - f(T(X)).mean()
        with torch.no_grad():
            T_loss_ar.append(T_loss.cpu().detach().numpy().mean())
        T_opt.zero_grad(); T_loss.backward(); T_opt.step()

    #f optimization
    T.eval(); f.train(True)
    X, Y = to_torch(sample_from_tensor(features_female)), to_torch(sample_from_tensor(features_male))
    f_loss = f(T(X)).mean() - (W * f(Y)).mean()
    with torch.no_grad():
            f_loss_ar.append(f_loss.cpu().detach().numpy())
    f_opt.zero_grad(); f_loss.backward(); f_opt.step()

    if step % 100 == 0:
        clear_output(wait=True)
        print("Step", step)
        print("T_loss", T_loss_ar)
        print("f_loss", f_loss_ar)




Step 40000
T_loss [np.float32(-38481.754), np.float32(-57704.44), np.float32(-83019.53), np.float32(-110111.664), np.float32(-132164.39), np.float32(-156703.25), np.float32(-175479.61), np.float32(-201466.28), np.float32(-222796.6), np.float32(-242773.48)]
f_loss [array(-10373579., dtype=float32)]


## Транспорт в новый домен

### Функция, которая заменяет каждый исходный вектор фичей на преобразованный

In [35]:
def xnot_transform(source_tensor):
    with torch.no_grad():
        result = torch.empty(source_tensor.shape[0],1024) #Initialize empty tensor with the same dimensions as an input feature vector
        for i in range(source_tensor.shape[0]):
            result[i] = T(source_tensor[i])

    return result


### Загружаем звуковую дорожку с женским голосом, которой не было в тренировочной выборке и формируем массив из векторов фичей с помощью WavLM

In [36]:
validation_path = "data/valid.flac" #Путь до файла с женским голосом для валидации
features_valid = xnotvc.get_features(validation_path) #Формируем массив из векторов фичей

In [37]:
print(features_valid.shape) #Исходный массив: 646 векторов размерностью 1024


torch.Size([646, 1024])


Формируем преобразованный массив

In [38]:
feautures_transformed = xnot_transform(features_valid) #Преобразованный массив

In [39]:
print(feautures_transformed.shape) #Размерность преобразованного массива должна совпасть с размерностью исходного = 646х1024

torch.Size([646, 1024])


In [40]:
print(feautures_transformed[5])

tensor([  4.6231, -15.2500,  14.5403,  ...,  10.0333,   6.1046,   5.0869])


## Вокодинг

Применяем метод для вокодинга фичей с помощью HiFi GAN

In [41]:
out_wav = xnotvc.vocode(feautures_transformed[None].to(DEVICE)).cpu().squeeze() #Вокодинг преобразованного с помощью XNOT вектора фичей
        

In [42]:
print(out_wav.shape)

torch.Size([206720])


In [43]:
import IPython.display as ipd

Примеч. У меня внутри ноутбука не выводит звук даже на оригинальных примерах от авторов knn-vc

In [44]:
ipd.Audio(out_wav.numpy(), rate=16000)

Сохраняем файл с аудиодорожкой

In [45]:
torchaudio.save('xnot_vc_test.flac', out_wav[None], 16000)

#### ***Вывод:*** На данный момент MLP не позволяет преобразовать исходные векторы фичей в целевые с помощью подхода XNOT. Вероятно, нужно использовать более сложную архитектуру, например CNN

## Проверка на обратный вокодинг неизмененного вектора фичей

In [32]:
out_wav_test = xnotvc.vocode(features_valid[None].to(DEVICE)).cpu().squeeze() #Вокодинг вектора фичей, полученного с помощью WavLM -> должны получить исходную аудиодорожку

In [33]:
print(out_wav_test.shape)

torch.Size([206720])


In [34]:
torchaudio.save('data/hifi_gan_test.flac', out_wav_test[None], 16000)

***Вывод:*** Hifi GAN преобразовал фичи исходной последовательности и восстановил звуковую дорожку -> работает и проблема в нейронных сетях на шаге XNOT